In [36]:
import pandas as pd

# Movie Dataset

In [37]:
df = pd.read_csv('movies_metadata.csv')

/var/folders/6q/q891flcj0r375hpjwrt2wtbm0000gn/T/ipykernel_72206/2089594497.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_metadata.csv')


In [38]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

# Text pre-processing

In [40]:
en_df = df[df['original_language'] == 'en']

In [41]:
en_df[['poster_path']].iloc[0]

poster_path    /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
Name: 0, dtype: object

In [42]:
en_df[[ 'tagline', 'original_title', 'overview']]

,tagline,original_title,overview
0,NaN,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Roll the dice and unleash the excitement!,Jumanji,When siblings Judy and Peter discover an encha...
2,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Friends are the people who let you be yourself...,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...,...
45459,NaN,Caged Heat 3000,It's the year 3000 AD. The world's most danger...
45460,NaN,Robin Hood,"Yet another version of the classic epic, with ..."
45463,A deadly game of wits.,Betrayal,"When one of her hits goes wrong, a professiona..."
45464,NaN,Satana likuyushchiy,"In a small town live two brothers, one a minis..."


In [43]:
en_df = en_df[[ 'genres', 'tagline', 'original_title', 'overview','vote_average','release_date' ]]

In [44]:
en_df['genres'].iloc[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [45]:
import ast

def extract_genre_names(genre_list_str):
    genre_list = ast.literal_eval(genre_list_str)
    genre_names = [genre['name'] for genre in genre_list]
    return ' '.join(genre_names)

# Apply the function to the genres column
en_df['genre_names'] = en_df['genres'].apply(extract_genre_names)

# Combine the genre names, tagline, original_title, and overview into a new column
en_df['combined_text'] = en_df.apply(
    lambda row: f"{row['original_title']}\n{row['overview']}\n{row['genre_names']}\n{row['tagline']}", axis=1
)


In [46]:
en_df['combined_text'].replace('nan', '', inplace=True)


# Prompt Engineering to Fill missing values

In [14]:
from openai import OpenAI
client = OpenAI(
      api_key='<Fill with openai key>'
  )

# def get_embedding(text, model="text-embedding-3-small"):
#    text = text.replace("\n", " ")
#    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
def generate_plot(title):
    response = client.completions.create(
        model="text-davinci-003",
        prompt=f"Provide a brief plot summary for the movie titled '{title}':",
        max_tokens=100
    )
    return response.choices[0].text.strip()
for index, row in en_df.iterrows():
    if pd.isna(row['overview']):
        title = row['title']
        en_df.at[index, 'overview'] = generate_plot(title)


# Upload to pinecone

In [47]:
from pinecone import Pinecone

pc = Pinecone(api_key="<API_KEY>")
index = pc.Index("movies-index")

In [57]:
# ! pip install -U sentence-transformers



In [48]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("Snowflake/snowflake-arctic-embed-m")

queries = ['what is snowflake?', 'Where can I get the best tacos?']
documents = ['The Data Cloud!', 'Mexico City of Course!']

query_embeddings = model.encode(queries, prompt_name="query")
document_embeddings = model.encode(documents)



In [49]:
en_df.columns

Index(['genres', 'tagline', 'original_title', 'overview', 'vote_average',
       'release_date', 'genre_names', 'combined_text'],
      dtype='object')

In [70]:
import numpy as np
def process_and_upsert_batch(batch_df):
    # Generate embeddings for the batch
    batch_df['embeddings'] = batch_df['combined_text'].apply(lambda x: model.encode(x))
    
    vectors = [
        {
            'id': str(idx),
            'values': embedding.tolist(),
            'metadata': {
                'tagline': row[2] if str(row[2])!='nan' else '',
                'original_title':  row[3] if str(row[3])!='nan' else '',
                'overview':  row[4] if str(row[4])!='nan' else '',
                'genre_names':  row[5] if str(row[5])!='nan' else '',
                'vote_average': row[6] if str(row[6])!='nan' else '',
                'release_date': row[7] if str(row[7])!='nan' else ''
            }
        }
        for idx, embedding, row in zip(batch_df.index, batch_df['embeddings'], batch_df.itertuples())
    ]
    print(vectors[0]['metadata'])
    # Upsert data to Pinecone
    index.upsert(vectors, namespace= "movie-desc")

# Process and upsert data in batches
batch_size = 5
for start in range(0, len(en_df), batch_size):
    end = start + batch_size
    batch_df = en_df.iloc[start:end]
    print(start, end)
    process_and_upsert_batch(batch_df)

In [56]:
import numpy as np
def process_and_upsert_batch(batch_df):
    for idx, row in zip(batch_df.index, batch_df.itertuples()):
        index.update(
            id=str(idx), 
            set_metadata={
                'tagline': row[2] if str(row[2])!='nan' else '',
                'original_title':  row[3] if str(row[3])!='nan' else '',
                'overview':  row[4] if str(row[4])!='nan' else '',
                'vote_average':  row[5] if str(row[5])!='nan' else '',
                'release_date': row[6] if str(row[6])!='nan' else '',
                'genre_names': row[7] if str(row[7])!='nan' else ''
            }, 
            namespace="movie-desc"
        )
    

# Process and upsert data in batches
batch_size = 5
for start in range(0, len(en_df), batch_size):
    end = start + batch_size
    batch_df = en_df.iloc[start:end]
    print(start, end)
    process_and_upsert_batch(batch_df)

In [83]:
model

(1024,
 SentenceTransformer(
   (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
   (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
   (2): Normalize()
 ))

In [85]:
model = SentenceTransformer("Snowflake/snowflake-arctic-embed-l")


# Test Retrieval

In [86]:
query_encode = model.encode("toy story", prompt_name="query").tolist()
print(query_encode)
ans = index.query(
            namespace="movie-desc",
            vector=query_encode,
            top_k=2,
            include_values=True,
            include_metadata=True,
        )

[0.0006205877871252596, -0.04039580374956131, -0.016616597771644592, -0.0186233501881361, -0.0017003162065520883, 0.0007085714605636895, 0.006175865884870291, 0.04279191792011261, -0.015176527202129364, 0.023550309240818024, 0.02057909220457077, 0.04330214858055115, 0.036801308393478394, 0.08008985221385956, -0.0764324739575386, 0.018336240202188492, 0.042035963386297226, -0.003686097450554371, 0.003639763919636607, -0.03559580817818642, 0.0660933181643486, -0.03403739258646965, -0.0607580728828907, 0.00994547177106142, 0.023384450003504753, -0.0055022891610860825, 0.026780685409903526, -0.052272532135248184, -0.03258457034826279, 0.010710720904171467, -0.02990863285958767, -0.019395167008042336, -0.008037189021706581, -0.00033168488880619407, 0.06579882651567459, 0.0029855261091142893, -0.007249683141708374, -0.022542675957083702, 0.05827019736170769, -0.030103323981165886, -0.002772154752165079, 0.03978709876537323, 0.016505075618624687, -0.0004118595679756254, 0.0016655598301440477,

In [87]:
ans['matches'][1]

{'id': '0',
 'metadata': {'genre_names': 'Animation Comedy Family',
              'original_title': 'Toy Story',
              'overview': "Led by Woody, Andy's toys live happily in his room "
                          "until Andy's birthday brings Buzz Lightyear onto "
                          "the scene. Afraid of losing his place in Andy's "
                          'heart, Woody plots against Buzz. But when '
                          'circumstances separate Buzz and Woody from their '
                          'owner, the duo eventually learns to put aside their '
                          'differences.',
              'tagline': ''},
 'score': 0.4054313,
 'values': [0.00103974901,
            -0.0438692495,
            0.0167565271,
            0.0105610816,
            -0.0107499,
            -0.00248907949,
            -0.00687012868,
            -0.0264384747,
            0.0121681402,
            -0.00028424809,
            0.0101636238,
            0.0477836281,
          

In [58]:
# ! pip install langchain_openai

In [12]:
import os
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings


os.environ['PINECONE_API_KEY'] = 'cc93780d-3526-4189-89d5-3cbafb9fb92a'

index_name = "movies-index"
embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-l")

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [16]:
query_result = embeddings.embed_query("toy story")
query_result


[0.003387298434972763,
 -0.06616361439228058,
 -0.03423302248120308,
 -0.009851091541349888,
 0.029259558767080307,
 0.005391408689320087,
 -0.004383192863315344,
 -0.025335753336548805,
 0.018641989678144455,
 -0.006520237307995558,
 0.040114566683769226,
 0.06167575344443321,
 0.03616737574338913,
 0.04914645478129387,
 -0.02859414555132389,
 -0.01295321062207222,
 0.02672037109732628,
 -0.017998330295085907,
 0.007978042587637901,
 -0.01228692103177309,
 0.04732171446084976,
 0.00017983072029892355,
 -0.0004479232884477824,
 -0.004884154070168734,
 0.019832978025078773,
 0.0004437743918970227,
 0.03347877413034439,
 -0.008819185197353363,
 -0.01602739840745926,
 0.036022212356328964,
 -0.020723605528473854,
 -0.014113516546785831,
 0.030884381383657455,
 0.00859671737998724,
 0.030449286103248596,
 -0.016804423183202744,
 -0.004328859969973564,
 -0.004179177340120077,
 0.021731654182076454,
 -0.049640875309705734,
 0.001866846694611013,
 0.023173972964286804,
 0.05476897582411766,
 

In [28]:

# ans = index.query(
#             namespace="movie-desc",
#             vector=query_result,
#             top_k=2,
#             include_values=True,
#             include_metadata=True,
#         )

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    api_key="sk-proj-aF6BXRqlyWt4neiE3foKT3BlbkFJiMzIaRwmBQ89RRR3fqZH",
    temperature=0.7,
    max_tokens=150
    )
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant specializing in movies. Use the below context only {context}"),
    ("user", "{input}")
])
chain = prompt | llm 

mess = chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": "toy story: Led by Woody, Andy's toys live happily in his room "
    })

In [32]:
mess.content

"I'm sorry, but it seems like your question is not related to the context provided about Toy Story. Could you please provide more details or rephrase your question so I can assist you better?"

# Backend logic test

In [35]:
llm = ChatOpenAI(
    api_key="",
    temperature=0.7,
    max_tokens=150
    )
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant specializing in movies."),
    ("user", """
     You are a helpful AI assistant. 
     Use the following pieces of context to answer the question at the end. 
     If you don't have enough information to answer the question, say "I don't have enough information to answer this question.
     
    Context: {context}

    Question: {question}

    Answer:
     """)
])

chain = prompt | llm 

mess = chain.invoke({
            "question": "toy story",
            "context": "story is good"
            })

print(mess)


content='Toy Story is a beloved animated film that follows the adventures of toys that come to life when their owner, Andy, is not around. It is known for its heartwarming story, engaging characters, and innovative animation techniques. Overall, Toy Story is considered a classic in the world of animated films.' response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 91, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-052b5a22-67e9-496d-90ba-c4dd6672eff0-0' usage_metadata={'input_tokens': 91, 'output_tokens': 60, 'total_tokens': 151}
